In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetV2B0, ResNet50, ConvNeXtTiny
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, LearningRateScheduler
from tensorflow.keras.losses import SparseCategoricalCrossentropy
import warnings

# Mixed precision for faster training and better generalization
tf.keras.mixed_precision.set_global_policy('mixed_float16')

warnings.filterwarnings('ignore')


In [2]:
# ============ Load Data ============
train_df = pd.read_csv('/kaggle/input/bttai-ajl-2025/train.csv')
test_df = pd.read_csv('/kaggle/input/bttai-ajl-2025/test.csv')

# Generate file paths
train_df['file_path'] = train_df.apply(
    lambda row: f"/kaggle/input/bttai-ajl-2025/train/train/{row['label']}/{row['md5hash']}.jpg", axis=1
)
test_df['file_path'] = test_df['md5hash'].apply(
    lambda x: f"/kaggle/input/bttai-ajl-2025/test/test/{x}.jpg"
)

# Remove invalid rows
train_df = train_df[(train_df['fitzpatrick_scale'] > 0) & (train_df['label'].notna())]
train_df = train_df[train_df['file_path'].apply(os.path.exists)]
test_df = test_df[test_df['file_path'].apply(os.path.exists)]

In [3]:
# ============ Encode Labels ============
label_encoder = LabelEncoder()
train_df['encoded_label'] = label_encoder.fit_transform(train_df['label'])

num_classes = len(label_encoder.classes_)

# ============ Compute Class Weights ============
class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(train_df['encoded_label']),
    y=train_df['encoded_label']
)
class_weights_dict = dict(enumerate(class_weights))

# ============ Learning Rate Scheduler ============
def cosine_decay_with_warmup(epoch, lr):
    warmup_epochs = 3
    if epoch < warmup_epochs:
        return lr * (epoch + 1) / warmup_epochs
    else:
        return 1e-4 * 0.5 * (1 + np.cos(np.pi * (epoch - warmup_epochs) / (50 - warmup_epochs)))


In [4]:
# ============ Mixup and Cutmix ============
def mixup(batch_images, batch_labels, alpha=0.4):
    lam = np.random.beta(alpha, alpha)
    index = np.random.permutation(batch_images.shape[0])
    mixed_images = lam * batch_images + (1 - lam) * batch_images[index]
    mixed_labels = lam * batch_labels + (1 - lam) * batch_labels[index]
    return mixed_images, mixed_labels

def cutmix(batch_images, batch_labels, alpha=0.4):
    lam = np.random.beta(alpha, alpha)
    h, w = batch_images.shape[1:3]
    r_x = np.random.uniform(0, w)
    r_y = np.random.uniform(0, h)
    r_w = w * np.sqrt(1 - lam)
    r_h = h * np.sqrt(1 - lam)

    x1 = int(np.clip(r_x - r_w / 2, 0, w))
    y1 = int(np.clip(r_y - r_h / 2, 0, h))
    x2 = int(np.clip(r_x + r_w / 2, 0, w))
    y2 = int(np.clip(r_y + r_h / 2, 0, h))

    index = np.random.permutation(batch_images.shape[0])
    batch_images[:, y1:y2, x1:x2, :] = batch_images[index, y1:y2, x1:x2, :]
    batch_labels = lam * batch_labels + (1 - lam) * batch_labels[index]

    return batch_images, batch_labels

In [5]:

# ============ Data Augmentation ============
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    brightness_range=[0.8, 1.2],
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = datagen.flow_from_dataframe(
    train_df,
    x_col='file_path',
    y_col='encoded_label',
    target_size=(512, 512),  # ✅ Increased size for better details
    batch_size=16,           # ✅ Smaller batch size for stability
    class_mode='raw'
)

test_datagen = ImageDataGenerator()
test_generator = test_datagen.flow_from_dataframe(
    test_df,
    x_col='file_path',
    target_size=(512, 512),
    batch_size=16,
    class_mode=None,
    shuffle=False
)

Found 2752 validated image filenames.
Found 1227 validated image filenames.


In [6]:
# ============ Build Model ============
def build_model(base_model):
    base_model.trainable = False
    x = GlobalAveragePooling2D()(base_model.output)
    x = Dropout(0.5)(x)
    output = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=output)
    return model



In [ ]:
# EfficientNetV2B0
model1 = build_model(EfficientNetV2B0(include_top=False, weights='imagenet', input_shape=(512, 512, 3)))
model1.compile(optimizer=Adam(learning_rate=1e-4), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model1.fit(train_generator, epochs=30, class_weight=class_weights_dict)
print("complete")

# ResNet50
model2 = build_model(ResNet50(include_top=False, weights='imagenet', input_shape=(512, 512, 3)))
model2.compile(optimizer=Adam(learning_rate=1e-4), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model2.fit(train_generator, epochs=30, class_weight=class_weights_dict)
print("complete")

# ConvNeXtTiny
model3 = build_model(ConvNeXtTiny(include_top=False, weights='imagenet', input_shape=(512, 512, 3)))
model3.compile(optimizer=Adam(learning_rate=1e-4), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model3.fit(train_generator, epochs=30, class_weight=class_weights_dict)
print("complete")

# ============ Model Averaging ============
pred1 = model1.predict(test_generator)
pred2 = model2.predict(test_generator)
pred3 = model3.predict(test_generator)

final_predictions = (0.4 * pred1) + (0.3 * pred2) + (0.3 * pred3)
test_df['label'] = label_encoder.inverse_transform(np.argmax(final_predictions, axis=1))

# ============ Create Submission ============
submission = test_df[['md5hash', 'label']]
submission.to_csv('/kaggle/working/sample_submission.csv', index=False)


24274472/24274472 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Epoch 1/30
172/172 ━━━━━━━━━━━━━━━━━━━━ 251s 1s/step - accuracy: 0.0552 - loss: 3.1580
Epoch 2/30
172/172 ━━━━━━━━━━━━━━━━━━━━ 211s 1s/step - accuracy: 0.0919 - loss: 2.9705
Epoch 3/30
113/172 ━━━━━━━━━━━━━━━━━━━━ 1:12 1s/step - accuracy: 0.1027 - loss: 2.9312